In [8]:
%pylab inline
from ipyparallel import Client, error
cluster=Client(profile="mpi")
view=cluster[:]
view.block=True

try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om

Populating the interactive namespace from numpy and matplotlib


# DOEDriver

DOEDriver facilitates performing a design of experiments (DOE) with your OpenMDAO model. It will run your model multiple times with different values for the design variables depending on the selected input generator. A number of generators are available, each with its own parameters that can be specified when it is instantiated:

  - [UniformGenerator](../../../_srcdocs/packages/drivers/doe_generators.html#openmdao.drivers.doe_generators.UniformGenerator)
  - [FullFactorialGenerator](../../../_srcdocs/packages/drivers/doe_generators.html#openmdao.drivers.doe_generators.FullFactorialGenerator)
  - [PlackettBurmanGenerator](../../../_srcdocs/packages/drivers/doe_generators.html#openmdao.drivers.doe_generators.PlackettBurmanGenerator)
  - [BoxBehnkenGenerator](../../../_srcdocs/packages/drivers/doe_generators.html#openmdao.drivers.doe_generators.BoxBehnkenGenerator)
  - [LatinHypercubeGenerator](../../../_srcdocs/packages/drivers/doe_generators.html#openmdao.drivers.doe_generators.LatinHypercubeGenerator)
  - [CSVGenerator](../../../_srcdocs/packages/drivers/doe_generators.html#openmdao.drivers.doe_generators.CSVGenerator)
  - [ListGenerator](../../../_srcdocs/packages/drivers/doe_generators.html#openmdao.drivers.doe_generators.ListGenerator)

```{Note}
`FullFactorialGenerator`, `PlackettBurmanGenerator`, `BoxBehnkenGenerator` and `LatinHypercubeGenerator` are provided via the [pyDOE2](https://pypi.org/project/pyDOE2/) package, which is an updated version of [pyDOE](https://pythonhosted.org/pyDOE/). See the original [pyDOE](https://pythonhosted.org/pyDOE/) page for information on those algorithms.
```

The generator instance may be supplied as an argument to the `DOEDriver` or as an option.

## DOEDriver Options

In [3]:
om.show_options_table("openmdao.drivers.doe_driver.DOEDriver")

Option,Default,Acceptable Values,Acceptable Types,Description
debug_print,[],N/A,['list'],"List of what type of Driver variables to print at each iteration. Valid items in list are 'desvars', 'ln_cons', 'nl_cons', 'objs', 'totals'"
generator,DOEGenerator,N/A,['DOEGenerator'],"The case generator. If default, no cases are generated."
procs_per_model,1,N/A,['int'],Number of processors to give each model under MPI.
run_parallel,False,"[True, False]",['bool'],Set to True to execute cases in parallel.


## DOEDriver Constructor

The call signature for the `DOEDriver` constructor is:

```{eval-rst}
    .. automethod:: openmdao.drivers.doe_driver.DOEDriver.__init__
       :noindex:
```  

## Simple Example

`UniformGenerator` implements the simplest method and will generate a requested number of samples randomly selected from a uniform distribution across the valid range for each design variable. This example demonstrates its use with a model built on the :ref:`Paraboloid<tutorial_paraboloid_analysis>` Component. An [SqliteRecorder](../../../recording/saving_data.ipynb) is used to capture the cases that were generated. We can see that that the model was evaluated at random values of x and y between -10 and 10, per the lower and upper bounds of those design variables.

In [4]:
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', Paraboloid(), promotes=['*'])

model.add_design_var('x', lower=-10, upper=10)
model.add_design_var('y', lower=-10, upper=10)
model.add_objective('f_xy')

prob.driver = om.DOEDriver(om.UniformGenerator(num_samples=5))
prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))

prob.setup()

prob.set_val('x', 0.0)
prob.set_val('y', 0.0)

prob.run_driver()
prob.cleanup()

cr = om.CaseReader("cases.sql")
cases = cr.list_cases('driver')


driver
rank0:DOEDriver_Uniform|0


driver
rank0:DOEDriver_Uniform|1


driver
rank0:DOEDriver_Uniform|2


driver
rank0:DOEDriver_Uniform|3


driver
rank0:DOEDriver_Uniform|4


In [5]:
print(len(cases))

5


In [6]:
values = []
for case in cases:
    outputs = cr.get_case(case).outputs
    values.append((outputs['x'], outputs['y'], outputs['f_xy']))

print("\n".join(["x: %5.2f, y: %5.2f, f_xy: %6.2f" % xyf for xyf in values]))

x:  7.52, y:  3.15, f_xy:  92.14
x:  1.86, y: -9.79, f_xy:  13.57
x: -8.63, y: -8.72, f_xy: 229.91
x: -0.14, y:  4.78, f_xy:  83.35
x: -9.66, y: -6.92, f_xy: 232.76


In [7]:
assert(len(cases) == 5)

## Running a DOE in Parallel

In a parallel processing environment, it is possible for `DOEDriver` to run cases concurrently. This is done by setting the `run_parallel` option to True as shown in the following example and running your script using MPI.

Here we are using the `FullFactorialGenerator` with 3 levels to generate inputs for our Paraboloid model. With two inputs, $3^2=9$ cases have been generated. In this case we are running on four processors and have specified `options['run_parallel']=True` to run cases on all available processors. The cases have therefore been split with 3 cases run on the first processor and 2 cases on each of the other processors.

Note that, when running in parallel, the `SqliteRecorder` will generate a separate case file for each processor on which cases are recorded. The case files will have a suffix indicating the recording rank and a message will be displayed indicating the file name, as seen in the example.

In [9]:
%%px

import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid

from mpi4py import MPI

prob = om.Problem()

prob.model.add_subsystem('comp', Paraboloid(), promotes=['x', 'y', 'f_xy'])
prob.model.add_design_var('x', lower=0.0, upper=1.0)
prob.model.add_design_var('y', lower=0.0, upper=1.0)
prob.model.add_objective('f_xy')

prob.driver = om.DOEDriver(om.FullFactorialGenerator(levels=3))
prob.driver.options['run_parallel'] = True
prob.driver.options['procs_per_model'] = 1

prob.driver.add_recorder(om.SqliteRecorder("cases.sql"))

prob.setup()
prob.run_driver()

[stdout:0] 
Note: SqliteRecorder is running on multiple processors. Cases from rank 0 are being written to cases.sql_0.
Note: Metadata is being recorded separately as cases.sql_meta.
[stdout:1] Note: SqliteRecorder is running on multiple processors. Cases from rank 1 are being written to cases.sql_1.
[stdout:2] Note: SqliteRecorder is running on multiple processors. Cases from rank 2 are being written to cases.sql_2.
[stdout:3] Note: SqliteRecorder is running on multiple processors. Cases from rank 3 are being written to cases.sql_3.


Out[0:43]: False

Out[1:43]: False

Out[2:43]: False

Out[3:43]: False

In [10]:
%%px

prob.cleanup()

print(MPI.COMM_WORLD.size)

[stdout:0] 4
[stdout:1] 4
[stdout:2] 4
[stdout:3] 4


In [11]:
%%px

# check recorded cases from each case file
rank = MPI.COMM_WORLD.rank
filename = "cases.sql_%d" % rank
print(filename)

[stdout:0] cases.sql_0
[stdout:1] cases.sql_1
[stdout:2] cases.sql_2
[stdout:3] cases.sql_3


In [12]:
%%px


# SqliteCaseReader will automatically look for cases.sql_meta if
# metadata_filename is not specified, but test by explicitly
# using it here.
cr = om.CaseReader(filename, metadata_filename = 'cases.sql_meta')
cases = cr.list_cases('driver')

[output:0]

driver
rank0:DOEDriver_FullFactorial|0


driver
rank0:DOEDriver_FullFactorial|1


driver
rank0:DOEDriver_FullFactorial|2


[output:1]

...

driver
rank2:DOEDriver_FullFactorial|0


driver
rank2:DOEDriver_FullFactorial|1


[output:3]

driver
rank3:DOEDriver_FullFactorial|0


driver
rank3:DOEDriver_FullFactorial|1


In [13]:
%%px

print(len(cases))

[stdout:0] 3
[stdout:1] 2
[stdout:2] 2
[stdout:3] 2


In [14]:
%%px

values = []
for case in cases:
    outputs = cr.get_case(case).outputs
    values.append((outputs['x'], outputs['y'], outputs['f_xy']))

print("\n"+"\n".join(["x: %5.2f, y: %5.2f, f_xy: %6.2f" % xyf for xyf in values]))

[stdout:0] 
x:  0.00, y:  0.00, f_xy:  22.00
x:  0.50, y:  0.50, f_xy:  23.75
x:  1.00, y:  1.00, f_xy:  27.00
[stdout:1] 
x:  0.50, y:  0.00, f_xy:  19.25
x:  1.00, y:  0.50, f_xy:  21.75
[stdout:2] 
x:  1.00, y:  0.00, f_xy:  17.00
x:  0.00, y:  1.00, f_xy:  31.00
[stdout:3] 
x:  0.00, y:  0.50, f_xy:  26.25
x:  0.50, y:  1.00, f_xy:  28.75


In [15]:
%%px

del cr

# Test for missing metadata db file error
try:
    cr_test = om.CaseReader(filename, metadata_filename = 'nonexistant_filename')
    found_metadata = True
except IOError:
    found_metadata = False

print(found_metadata, "No error from SqliteCaseReader for missing metadata file.")


[stdout:0] False No error from SqliteCaseReader for missing metadata file.
[stdout:1] False No error from SqliteCaseReader for missing metadata file.
[stdout:2] False No error from SqliteCaseReader for missing metadata file.
[stdout:3] False No error from SqliteCaseReader for missing metadata file.


In [18]:
%%px

assert(len(cases) == 3 if rank == 0 else 2)
expected = [
    {'x': np.array([0.]), 'y': np.array([0.]), 'f_xy': np.array([22.00])},
    {'x': np.array([.5]), 'y': np.array([0.]), 'f_xy': np.array([19.25])},
    {'x': np.array([1.]), 'y': np.array([0.]), 'f_xy': np.array([17.00])},

    {'x': np.array([0.]), 'y': np.array([.5]), 'f_xy': np.array([26.25])},
    {'x': np.array([.5]), 'y': np.array([.5]), 'f_xy': np.array([23.75])},
    {'x': np.array([1.]), 'y': np.array([.5]), 'f_xy': np.array([21.75])},

    {'x': np.array([0.]), 'y': np.array([1.]), 'f_xy': np.array([31.00])},
    {'x': np.array([.5]), 'y': np.array([1.]), 'f_xy': np.array([28.75])},
    {'x': np.array([1.]), 'y': np.array([1.]), 'f_xy': np.array([27.00])},
]

assert("\n"+"\n".join(["x: %5.2f, y: %5.2f, f_xy: %6.2f" % xyf for xyf in values]) == expected)

CompositeError: one or more exceptions from call to method: execute
[0:execute]: AssertionError: 
[1:execute]: AssertionError: 
[2:execute]: AssertionError: 
[3:execute]: AssertionError: 